In [1]:
import yaml
import numpy as np
import pandas as pd
from timeit import default_timer as timer
from datetime import timedelta, date

from macrosynergy.management.dq import DataQueryInterface

import os
print(f"Current working directory: {os.getcwd()}")
path_to_config = '..//data//'

Current working directory: /Users/kurransteeds/repos/macrosynergy


In [2]:
cids_dmca = ['AUD', 'CAD', 'CHF', 'EUR', 'GBP', 'JPY', 'NOK', 'NZD', 'SEK', 'USD']  # DM currency areas
cids_dmec = ['DEM', 'ESP', 'FRF', 'ITL', 'NLG']  # DM euro area countries
cids_latm = ['ARS', 'BRL', 'COP', 'CLP', 'MXN', 'PEN']  # Latam countries
cids_emea = ['HUF', 'ILS', 'PLN', 'RON', 'RUB', 'TRY', 'ZAR']  # EMEA countries
cids_emas = ['CNY', 'HKD', 'IDR', 'INR', 'KRW', 'MYR', 'PHP', 'SGD', 'THB', 'TWD']  # EM Asia countries
cids_dm = cids_dmca + cids_dmec
cids_em = cids_latm + cids_emea + cids_emas
cids = sorted(cids_dm + cids_em)

cids_eufx = ['CHF', 'HUF', 'NOK', 'PLN', 'RON', 'SEK']  # EUR benchmark
cids_g2fx = ['GBP', 'RUB', 'TRY']  # dual benchmark
cids_usfx = ['AUD', 'BRL', 'CAD', 'CLP', 'CNY', 'COP', 'EUR', 'IDR', 'ILS', 'INR', 'JPY', 'KRW', 'MYR',
             'MXN', 'NZD', 'PEN', 'PHP', 'SGD', 'THB', 'TWD', 'ZAR']  # USD benchmark
cids_fx = cids_usfx + cids_eufx + cids_g2fx

In [3]:
def dq_tickers(tickers, metrics=['value'], start_date = '2000-01-01'):
    
    """
    Returns standardized dataframe of specified base tickers and metric
    
    
    :param <List[str]> tickers: JPMaQS ticker of form <cid>_<xcat>.
    :param <List[str]> metrics: must choose one or more from 'value', 'eop_lag', 'mop_lag', or 'grade'. Default is ['value'].
    :param <str> start_date: first date in ISO 8601 string format. 
    :param <str> path: relative path from notebook to credential files.
    
    :return <pd.Dataframe> standardized dataframe with columns 'cid', 'xcats', 'real_date' and chosen metrics.
    """
    
    with open(f"config.yml", 'r') as f:
        cf = yaml.load(f, Loader=yaml.FullLoader)

    with DataQueryInterface(
                username=cf["dq"]["username"],
                password=cf["dq"]["password"],
                crt=f"api_macrosynergy_com.crt",
                key=f"api_macrosynergy_com.key") as dq:
        
        df = dq.get_ts_expression(expression=tickers, original_metrics=metrics, start_date=start_date, bool_=False)
        
        if isinstance(df, pd.DataFrame):
            df = df.sort_values(['cid', 'xcat', 'real_date']).reset_index(drop=True)

    return df

In [4]:
def dq_download(tickers=None, xcats=None, cids=None, metrics=['value'], start_date = '2000-01-01', path = '..//data//'):
    
    """
    Returns standardized dataframe of specified base tickers and metrics
        
    :param <List[str]> tickers: JPMaQS ticker of form <cid>_<xcat>. Can be combined with selection of categories.
    :param <List[str]> xcats: JPMaQS category codes. Downloaded for all standard cross sections identifiers available 
    (if cids are not specified) or those selected (if cids are specified).
    Standard cross sections here include major developed and emerging currency markets. See JPMaQS documentation.
    :param <List[str]> cids: JPMaQS cross-section identifiers, typically based  on currency code. See JPMaQS documentation.
    :param <str> metrics: must choose one or more from 'value', 'eop_lag', 'mop_lag', or 'grade'. Default is ['value'].
    # Todo: allow multiple metrics to be downloaded
    :param <str> start_date: first date in ISO 8601 string format. 
    :param <str> path: relative path from notebook to credential files.
    
    :return <pd.Dataframe> standardized dataframe with columns 'cid', 'xcats', 'real_date' and chosen metrics.
    """
    
    # A. Collect all standard cross sections.
    
    if (cids is None) & (xcats is not None):
    
        cids_dmca = ['AUD', 'CAD', 'CHF', 'EUR', 'GBP', 'JPY', 'NOK', 'NZD', 'SEK', 'USD']  # DM currency areas
        cids_dmec = ['DEM', 'ESP', 'FRF', 'ITL', 'NLG']  # DM euro area countries
        cids_latm = ['BRL', 'COP', 'CLP', 'MXN', 'PEN']  # Latam countries
        cids_emea = ['HUF', 'ILS', 'PLN', 'RON', 'RUB', 'TRY', 'ZAR']  # EMEA countries
        cids_emas = ['CNY', 'IDR', 'INR', 'KRW', 'MYR', 'PHP', 'SGD', 'THB', 'TWD']  # EM Asia countries
        cids_dm = cids_dmca + cids_dmec
        cids_em = cids_latm + cids_emea + cids_emas
        cids = sorted(cids_dm + cids_em)  # standard default
        
    
    # B. Collect all tickers and metrics to be downloaded
    
    if isinstance(tickers, str):
        tickers = [tickers]
    if isinstance(xcats, str):
        xcats = [xcats]
    if isinstance(cids, str):
        cids = [cids]
    if isinstance(metrics, str):
        metrics = [metrics]
     
    if tickers is None:
        tickers = []
    
    print(f"xcats: {xcats}")
    if xcats is not None:
        add_tix = [cid + '_' + xcat for xcat in xcats for cid in cids]
        tickers = tickers + add_tix
    
    df = dq_tickers(tickers, metrics=metrics, start_date = start_date)
        
    return df

In [5]:
tix = [cid + '_CPIXFE_SJA_P6M6ML6AR' for cid in cids]

start = timer()
df = dq_download(tickers=tix, metrics=['value'], xcats = None, start_date='2021-11-01')  # import via API
end = timer()
print(df)

print("Download time from DQ: "+ str(timedelta(seconds=end - start)))
print("Last updated:", date.today())

xcats: None
The ticker, DB(JPMAQS,ARS_CPIXFE_SJA_P6M6ML6AR, does not exist in the Database.
The ticker, DB(JPMAQS,ITL_CPIXFE_SJA_P6M6ML6AR, does not exist in the Database.
The ticker, DB(JPMAQS,DEM_CPIXFE_SJA_P6M6ML6AR, does not exist in the Database.
The ticker, DB(JPMAQS,ESP_CPIXFE_SJA_P6M6ML6AR, does not exist in the Database.
The ticker, DB(JPMAQS,NLG_CPIXFE_SJA_P6M6ML6AR, does not exist in the Database.
The ticker, DB(JPMAQS,CLP_CPIXFE_SJA_P6M6ML6AR, does not exist in the Database.
The ticker, DB(JPMAQS,HKD_CPIXFE_SJA_P6M6ML6AR, does not exist in the Database.
The ticker, DB(JPMAQS,FRF_CPIXFE_SJA_P6M6ML6AR, does not exist in the Database.
     cid                  xcat  real_date     value
0    AUD  CPIXFE_SJA_P6M6ML6AR 2021-11-01  1.820391
1    AUD  CPIXFE_SJA_P6M6ML6AR 2021-11-02  1.820391
2    AUD  CPIXFE_SJA_P6M6ML6AR 2021-11-03  1.820391
3    AUD  CPIXFE_SJA_P6M6ML6AR 2021-11-04  1.820391
4    AUD  CPIXFE_SJA_P6M6ML6AR 2021-11-05  1.820391
..   ...                   ...      

In [6]:
print(df.shape)
print(df.head())
print(df['cid'].unique())
print(df['xcat'].unique())

(330, 4)
   cid                  xcat  real_date     value
0  AUD  CPIXFE_SJA_P6M6ML6AR 2021-11-01  1.820391
1  AUD  CPIXFE_SJA_P6M6ML6AR 2021-11-02  1.820391
2  AUD  CPIXFE_SJA_P6M6ML6AR 2021-11-03  1.820391
3  AUD  CPIXFE_SJA_P6M6ML6AR 2021-11-04  1.820391
4  AUD  CPIXFE_SJA_P6M6ML6AR 2021-11-05  1.820391
['AUD' 'BRL' 'CAD' 'CHF' 'CNY' 'COP' 'EUR' 'GBP' 'HUF' 'IDR' 'ILS' 'INR'
 'JPY' 'KRW' 'MXN' 'MYR' 'NOK' 'NZD' 'PEN' 'PHP' 'PLN' 'RON' 'RUB' 'SEK'
 'SGD' 'THB' 'TRY' 'TWD' 'USD' 'ZAR']
['CPIXFE_SJA_P6M6ML6AR']
